In [2]:
from pyspark.sql import SparkSession
import email
from email.mime.text import MIMEText
from email.parser import Parser
import mailparser

#related to listing out directory files. 
from os import walk
from os import listdir
from os.path import isfile, join

import pandas as pd

In [3]:
spark = SparkSession.builder.appName('Ops').getOrCreate()
data_dir = '../data/skilling-j/'
max_int = 999999

Creating dataframes to explore each folder. 

I have tried to pick emails with a large amount of content in each folder to understand how the emails are structured. 
There also seems to be emails that are missing in the sequence. 

In [4]:
df_sent = spark.read.text(data_dir + "sent/7.")
df_sent_mail = spark.read.text(data_dir + "_sent_mail/7.")
df_sent_items = spark.read.text(data_dir + "sent_items/5.")
df_inbox = spark.read.text(data_dir + "inbox/3.")
df_notes_inbox = spark.read.text(data_dir + "notes_inbox/10.")
df_all_documents = spark.read.text(data_dir + "all_documents/7.")
df_deleted_items = spark.read.text(data_dir + "deleted_items/3.")
df_discussion_threads = spark.read.text(data_dir + "discussion_threads/6.")
df_mark = spark.read.text(data_dir + "mark/10.")

Mime-Verison: 1.0: https://en.wikipedia.org/wiki/MIME

In [5]:
df_sent.show(max_int,truncate=False)

+----------------------------------------------------------------------------+
|value                                                                       |
+----------------------------------------------------------------------------+
|Message-ID: <21711674.1075840099910.JavaMail.evans@thyme>                   |
|Date: Mon, 27 Dec 1999 02:53:00 -0800 (PST)                                 |
|From: katherine.brown@enron.com                                             |
|To: pam.benson@enron.com                                                    |
|Subject: Re: Meeting with Chairman of The Williams Company                  |
|Cc: sherri.reinartz@enron.com                                               |
|Mime-Version: 1.0                                                           |
|Content-Type: text/plain; charset=us-ascii                                  |
|Content-Transfer-Encoding: 7bit                                             |
|Bcc: sherri.reinartz@enron.com                     

In [6]:
df_sent_mail.show(max_int,truncate=False)

+------------------------------------------------------------------------------+
|value                                                                         |
+------------------------------------------------------------------------------+
|Message-ID: <14407572.1075840103637.JavaMail.evans@thyme>                     |
|Date: Thu, 7 Dec 2000 08:36:00 -0800 (PST)                                    |
|From: sherri.sera@enron.com                                                   |
|To: nkelley@tqcap.com                                                         |
|Subject: RE: Genesis Park                                                     |
|Mime-Version: 1.0                                                             |
|Content-Type: text/plain; charset=us-ascii                                    |
|Content-Transfer-Encoding: 7bit                                               |
|X-From: Sherri Sera                                                           |
|X-To: Neil Kelley <NKelley@

In [7]:
df_sent_items.show(max_int,truncate=False)

+--------------------------------------------------------------+
|value                                                         |
+--------------------------------------------------------------+
|Message-ID: <30255134.1075840163074.JavaMail.evans@thyme>     |
|Date: Tue, 15 May 2001 14:09:00 -0700 (PDT)                   |
|From: jeff.skilling@enron.com                                 |
|To: joannie.williamson@enron.com                              |
|Subject: test                                                 |
|Mime-Version: 1.0                                             |
|Content-Type: text/plain; charset=us-ascii                    |
|Content-Transfer-Encoding: 7bit                               |
|X-From: Jeff Skilling                                         |
|X-To: Joannie Williamson <Joannie Williamson/Corp/Enron@ENRON>|
|X-cc:                                                         |
|X-bcc:                                                        |
|X-Folder: \jskillin\Sent

In [8]:
df_inbox.show(max_int,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+---------------------------------------------------------

In [9]:
df_notes_inbox.show(max_int,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Message-ID: <1815387.1075840095926.JavaMail.evans@thyme>                                                                                                             |
|Date: Wed, 13 Dec 2000 07:58:00 -0800 (PST)                                                                                                                          |
|From: karen.denne@enron.com                                                                                                                                    

In [10]:
df_all_documents.show(max_int,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                          

In [11]:
df_deleted_items.show(max_int,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                     |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Message-ID: <19400385.1075852645722.JavaMail.evans@thyme>                                                                                                                                 |
|Date: Thu, 30 Aug 2001 08:30:54 -0700 (PDT)                                                                                                                                               |
|From: update@barescapital.com                         

In [12]:
df_discussion_threads.show(max_int,truncate=False)

+------------------------------------------------------------------------+
|value                                                                   |
+------------------------------------------------------------------------+
|Message-ID: <20387174.1075840081741.JavaMail.evans@thyme>               |
|Date: Mon, 1 Mar 1999 06:38:00 -0800 (PST)                              |
|From: markskilling@hotmail.com                                          |
|To: jeff.skilling@enron.com                                             |
|Subject:                                                                |
|Mime-Version: 1.0                                                       |
|Content-Type: text/plain; charset=us-ascii                              |
|Content-Transfer-Encoding: 7bit                                         |
|X-From: "Mark Skilling" <markskilling@hotmail.com>                      |
|X-To: Jeff Skilling                                                     |
|X-cc:                   

In [13]:
df_mark.show(max_int,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                       

I'll attempt to parse the email and header information.

In [14]:
headers = Parser().parse(open(data_dir + "sent/7.", 'r'))
for key, val in headers.items():
    print(key, "=>", val)

Message-ID => <21711674.1075840099910.JavaMail.evans@thyme>
Date => Mon, 27 Dec 1999 02:53:00 -0800 (PST)
From => katherine.brown@enron.com
To => pam.benson@enron.com
Subject => Re: Meeting with Chairman of The Williams Company
Cc => sherri.reinartz@enron.com
Mime-Version => 1.0
Content-Type => text/plain; charset=us-ascii
Content-Transfer-Encoding => 7bit
Bcc => sherri.reinartz@enron.com
X-From => Katherine Brown
X-To => Pam Benson
X-cc => Sherri Reinartz
X-bcc => 
X-Folder => \Jeffrey_Skilling_Dec2000\Notes Folders\Sent
X-Origin => SKILLING-J
X-FileName => jskillin.nsf


Remaining would be the body of the message, as well as all forwarded emails. I'll try to grab and parse this as well. 

In [15]:
f = data_dir + "sent/7."
mail = mailparser.parse_from_file(f)
mail.headers

{'Message-ID': '<21711674.1075840099910.JavaMail.evans@thyme>',
 'Date': 'Mon, 27 Dec 1999 02:53:00 -0800 (PST)',
 'From': 'katherine.brown@enron.com',
 'To': 'pam.benson@enron.com',
 'Subject': 'Re: Meeting with Chairman of The Williams Company',
 'Cc': 'sherri.reinartz@enron.com',
 'Mime-Version': '1.0',
 'Content-Type': 'text/plain; charset=us-ascii',
 'Content-Transfer-Encoding': '7bit',
 'Bcc': 'sherri.reinartz@enron.com',
 'X-From': 'Katherine Brown',
 'X-To': 'Pam Benson',
 'X-cc': 'Sherri Reinartz',
 'X-bcc': '',
 'X-Folder': '\\Jeffrey_Skilling_Dec2000\\Notes Folders\\Sent',
 'X-Origin': 'SKILLING-J',
 'X-FileName': 'jskillin.nsf'}

In [16]:
mail.body

'Pam, I was going through Jeff\'s email and found this.  See below - it looks \nlike this went to Catherine Brown of EES (my name is with a "K").  It looks \nlike Jeff would be available from 2:00 to 3:00.  I will put it on his \ncalendar to attend for one hour.  Is that good for you? \nKatherine\n\n\n\n\nPam Benson@ENRON_DEVELOPMENT\n12/27/99 09:38 AM\nTo: Stanley Horton@ENRON, Jeff Skilling@ENRON\ncc: Cindy Stark@ENRON, Sherri Reinartz@ENRON, Catherine Brown@ees \n\nSubject: Meeting with Chairman of The Williams Company\n\nJoe Sutton would like to invite you to attend a meeting with Keith Bailey, \nChairman of The Williams Company on January 5 from 2-4 pm.  \n\nThanks - Pam'

The following code allows us to seperate out the body, from all the forwarded chain. 

In [24]:
test = []

b = email.message_from_string(mail.body)
if b.is_multipart():
    for payload in b.get_payload():
        test.append(payload.get_payload())
        #print(payload.get_payload())
else:
    print(b.get_payload())
    


test not appendeD?
Pam, I was going through Jeff's email and found this.  See below - it looks 
like this went to Catherine Brown of EES (my name is with a "K").  It looks 
like Jeff would be available from 2:00 to 3:00.  I will put it on his 
calendar to attend for one hour.  Is that good for you? 
Katherine




Pam Benson@ENRON_DEVELOPMENT
12/27/99 09:38 AM
To: Stanley Horton@ENRON, Jeff Skilling@ENRON
cc: Cindy Stark@ENRON, Sherri Reinartz@ENRON, Catherine Brown@ees 

Subject: Meeting with Chairman of The Williams Company

Joe Sutton would like to invite you to attend a meeting with Keith Bailey, 
Chairman of The Williams Company on January 5 from 2-4 pm.  

Thanks - Pam


The following code is related to exploring the maildir

In [27]:
dirs = []
files = []

for (dirpath, dirnames, filenames) in walk(data_dir, followlinks=False):
    files.extend(filenames)
    print(dirpath)
    print(dirnames)
    print(filenames)

../data/skilling-j/
['sent_items', '_sent_mail', 'mark', 'inbox', 'discussion_threads', 'notes_inbox', 'deleted_items', 'sent', 'contacts', 'all_documents', 'calendar']
[]
../data/skilling-j/sent_items
[]
['1.', '16.', '110.', '28.', '5.', '179.', '22.', '46.', '35.', '38.', '47.', '4.', '39.', '23.', '19.', '48.', '11.', '37.', '105.', '26.', '8.', '14.', '9.', '31.', '42.', '36.', '10.', '41.', '12.', '17.', '104.', '106.', '3.', '33.', '43.', '25.', '50.', '18.', '13.', '44.', '32.', '27.', '45.', '21.', '51.', '24.', '15.', '111.', '40.', '49.', '20.', '34.', '103.', '2.']
../data/skilling-j/_sent_mail
[]
['346.', '305.', '1.', '368.', '309.', '377.', '310.', '321.', '16.', '308.', '110.', '133.', '335.', '74.', '88.', '272.', '124.', '320.', '289.', '323.', '78.', '284.', '28.', '125.', '5.', '75.', '341.', '399.', '326.', '400.', '22.', '373.', '401.', '285.', '95.', '279.', '394.', '46.', '118.', '333.', '315.', '294.', '56.', '35.', '288.', '38.', '270.', '6.', '47.', '404.', '

Following code lists out files for each dir inside a persons maildir.

In [37]:
onlydirs = listdir(data_dir)
#onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]

for folder in onlydirs:
    onlyfiles = [f for f in listdir(data_dir + folder) if isfile(join(data_dir + folder, f))]
    print(folder)
    print(onlyfiles)

sent_items
['1.', '16.', '110.', '28.', '5.', '179.', '22.', '46.', '35.', '38.', '47.', '4.', '39.', '23.', '19.', '48.', '11.', '37.', '105.', '26.', '8.', '14.', '9.', '31.', '42.', '36.', '10.', '41.', '12.', '17.', '104.', '106.', '3.', '33.', '43.', '25.', '50.', '18.', '13.', '44.', '32.', '27.', '45.', '21.', '51.', '24.', '15.', '111.', '40.', '49.', '20.', '34.', '103.', '2.']
_sent_mail
['346.', '305.', '1.', '368.', '309.', '377.', '310.', '321.', '16.', '308.', '110.', '133.', '335.', '74.', '88.', '272.', '124.', '320.', '289.', '323.', '78.', '284.', '28.', '125.', '5.', '75.', '341.', '399.', '326.', '400.', '22.', '373.', '401.', '285.', '95.', '279.', '394.', '46.', '118.', '333.', '315.', '294.', '56.', '35.', '288.', '38.', '270.', '6.', '47.', '404.', '338.', '378.', '76.', '340.', '375.', '311.', '93.', '286.', '287.', '96.', '112.', '59.', '371.', '325.', '4.', '121.', '39.', '23.', '19.', '91.', '328.', '292.', '397.', '350.', '330.', '55.', '385.', '48.', '128.

Now add to pandas dataframe. 

In [ ]:

df = pd.DataFrame()

Things below seemed to be ideas that did not work

In [1]:
enron_df = pd.DataFrame(columns=['Employee',
                                'Folder',
                                'Email-Name',
                                'Message-ID',
                                'Date',
                                'From',
                                'To',
                                'Subject',
                                'Cc',
                                'Mime-Version',
                                'Content-Type',
                                'Content-Transfer-Encoding',
                                'Bcc',
                                'X-From',
                                'X-To',
                                'X-cc',
                                'X-bcc',
                                'X-Folder',
                                'X-Origin',
                                'X-FileName'
                                'Body'
                                'Fwd_chain'])

NameError: name 'pd' is not defined

In [ ]:
#The following strips out the forwarded portion of the email
fwds = []
#Seperate the body of the email. This current attempt seems to dump everything but the headers into the body. Doesn't work so far.
if full_body.is_multipart():
        for payload in full_body.get_payload():
            fwds.append(payload.get_payload())
        mail_body = fwds[0]
        fwds.remove(0)
    else:
    "Fwd_chain": fwds

Observations:

1. Seems like some people's inboxes has even more folders inside. For example:
/home/kirt/Documents/ML_Enron_email_summary/data/maildir/geaccone-t/inbox/adaytum

To help simplify loading this data into pandas, I moved these folders up one level so it gets treated as it's own archive mailbox. 

